# Stanford Dogs CNN 

In [44]:
# basic modeling
import os
import torch
import torchvision
import torch.nn as nn
import numpy as np

# for reading and displaying images and editing
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline

# Loss Function
import torch.nn.functional as F
import torch.optim as optim

# Data Cleaning
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.transforms import ToTensor

In [45]:
# Directory where data is
data_dir = r'C:\Users\dsk02\Desktop\python_projects\torch_dogs\main_images\Images'
print(os.listdir(data_dir))

['n02085620-Chihuahua', 'n02085782-Japanese_spaniel', 'n02085936-Maltese_dog', 'n02086079-Pekinese', 'n02086240-Shih-Tzu', 'n02086646-Blenheim_spaniel', 'n02086910-papillon', 'n02087046-toy_terrier', 'n02087394-Rhodesian_ridgeback', 'n02088094-Afghan_hound', 'n02088238-basset', 'n02088364-beagle', 'n02088466-bloodhound', 'n02088632-bluetick', 'n02089078-black-and-tan_coonhound', 'n02089867-Walker_hound', 'n02089973-English_foxhound', 'n02090379-redbone', 'n02090622-borzoi', 'n02090721-Irish_wolfhound', 'n02091032-Italian_greyhound', 'n02091134-whippet', 'n02091244-Ibizan_hound', 'n02091467-Norwegian_elkhound', 'n02091635-otterhound', 'n02091831-Saluki', 'n02092002-Scottish_deerhound', 'n02092339-Weimaraner', 'n02093256-Staffordshire_bullterrier', 'n02093428-American_Staffordshire_terrier', 'n02093647-Bedlington_terrier', 'n02093754-Border_terrier', 'n02093859-Kerry_blue_terrier', 'n02093991-Irish_terrier', 'n02094114-Norfolk_terrier', 'n02094258-Norwich_terrier', 'n02094433-Yorkshire_t

In [46]:
# clean up classes name
temp = os.listdir(data_dir)

classes = []

# class cleaning

for each_folder in temp:
    
    # split words on -
    split_words = each_folder.split('-')
    
    # if there is more than one dash
    if len(split_words) > 2:
        
        # create temp word
        temp = ''
        # for each value in the split_words array (starting from 1)
        for i in range(1,len(split_words)):
            
            # if it's not the end add word + space
            if i != (len(split_words)-1):
                temp += (split_words[i] + ' ')
            else:
                # if end just add the word
                temp += (split_words[i])
        
        # append temp to classes
        classes.append(temp.lower())
    
    # if it equals 2 it's just nasty title + name, append name
    elif len(split_words) == 2:
        classes.append(each_folder.split('-')[1].lower())
    
    # outliers append 
    else:
        classes.append(each_folder.lower())
    
# print first few classes, all lower case
print(classes)

['chihuahua', 'japanese_spaniel', 'maltese_dog', 'pekinese', 'shih tzu', 'blenheim_spaniel', 'papillon', 'toy_terrier', 'rhodesian_ridgeback', 'afghan_hound', 'basset', 'beagle', 'bloodhound', 'bluetick', 'black and tan_coonhound', 'walker_hound', 'english_foxhound', 'redbone', 'borzoi', 'irish_wolfhound', 'italian_greyhound', 'whippet', 'ibizan_hound', 'norwegian_elkhound', 'otterhound', 'saluki', 'scottish_deerhound', 'weimaraner', 'staffordshire_bullterrier', 'american_staffordshire_terrier', 'bedlington_terrier', 'border_terrier', 'kerry_blue_terrier', 'irish_terrier', 'norfolk_terrier', 'norwich_terrier', 'yorkshire_terrier', 'wire haired_fox_terrier', 'lakeland_terrier', 'sealyham_terrier', 'airedale', 'cairn', 'australian_terrier', 'dandie_dinmont', 'boston_bull', 'miniature_schnauzer', 'giant_schnauzer', 'standard_schnauzer', 'scotch_terrier', 'tibetan_terrier', 'silky_terrier', 'soft coated_wheaten_terrier', 'west_highland_white_terrier', 'lhasa', 'flat coated_retriever', 'cur

In [47]:
from collections import defaultdict

# create lookup dictionary for index -> class
lookup = [x for x in range(0,len(classes))]

# initialize with strings
classLookup = defaultdict(str)

# index and class, push into defaultDict ->
for i, each_class in enumerate(classes):
    classLookup[i] = each_class
    
# will use at end for lookups on guesses
print(classLookup)

defaultdict(<class 'str'>, {0: 'chihuahua', 1: 'japanese_spaniel', 2: 'maltese_dog', 3: 'pekinese', 4: 'shih tzu', 5: 'blenheim_spaniel', 6: 'papillon', 7: 'toy_terrier', 8: 'rhodesian_ridgeback', 9: 'afghan_hound', 10: 'basset', 11: 'beagle', 12: 'bloodhound', 13: 'bluetick', 14: 'black and tan_coonhound', 15: 'walker_hound', 16: 'english_foxhound', 17: 'redbone', 18: 'borzoi', 19: 'irish_wolfhound', 20: 'italian_greyhound', 21: 'whippet', 22: 'ibizan_hound', 23: 'norwegian_elkhound', 24: 'otterhound', 25: 'saluki', 26: 'scottish_deerhound', 27: 'weimaraner', 28: 'staffordshire_bullterrier', 29: 'american_staffordshire_terrier', 30: 'bedlington_terrier', 31: 'border_terrier', 32: 'kerry_blue_terrier', 33: 'irish_terrier', 34: 'norfolk_terrier', 35: 'norwich_terrier', 36: 'yorkshire_terrier', 37: 'wire haired_fox_terrier', 38: 'lakeland_terrier', 39: 'sealyham_terrier', 40: 'airedale', 41: 'cairn', 42: 'australian_terrier', 43: 'dandie_dinmont', 44: 'boston_bull', 45: 'miniature_schnau

In [62]:
# load train images
train_img = []
train_targets = []

temp = os.listdir(data_dir)

# for each folder in the train set
for i_, each_folder in enumerate(temp):
    
    #current image names
    image_names = os.listdir(data_dir + '/' + str(temp[i_]))
    print(image_names)
    
    # for each file in the folder 
    for i, img_name in enumerate(image_names):
        
        # image path
        image_path = str(str(data_dir) + '/' + str(temp[i_]) + '/' + str(img_name))
        
        #read image
        img = imread(image_path)

        #change image shape to -> 3,28,28 (originally were massive)
        img = resize(img, (3,28,28))

        # convert dt -> may have to change if 2 big
        img = img.astype('float32')

        #append to trainning list
        train_img.append(img)

        #append to targets list
        train_targets.append(i)

['n02085620_10074.jpg', 'n02085620_10131.jpg', 'n02085620_10621.jpg', 'n02085620_1073.jpg', 'n02085620_10976.jpg', 'n02085620_11140.jpg', 'n02085620_11238.jpg', 'n02085620_11258.jpg', 'n02085620_11337.jpg', 'n02085620_11477.jpg', 'n02085620_1152.jpg', 'n02085620_11696.jpg', 'n02085620_11818.jpg', 'n02085620_11948.jpg', 'n02085620_1205.jpg', 'n02085620_12101.jpg', 'n02085620_12334.jpg', 'n02085620_1235.jpg', 'n02085620_1271.jpg', 'n02085620_12718.jpg', 'n02085620_1298.jpg', 'n02085620_13151.jpg', 'n02085620_1321.jpg', 'n02085620_13383.jpg', 'n02085620_1346.jpg', 'n02085620_13964.jpg', 'n02085620_14252.jpg', 'n02085620_14413.jpg', 'n02085620_14516.jpg', 'n02085620_1455.jpg', 'n02085620_1492.jpg', 'n02085620_1502.jpg', 'n02085620_1558.jpg', 'n02085620_1569.jpg', 'n02085620_1617.jpg', 'n02085620_1620.jpg', 'n02085620_1765.jpg', 'n02085620_1816.jpg', 'n02085620_1862.jpg', 'n02085620_1916.jpg', 'n02085620_199.jpg', 'n02085620_2053.jpg', 'n02085620_2188.jpg', 'n02085620_2204.jpg', 'n02085620_

KeyboardInterrupt: 

In [63]:
# convert to numpy arrays, make sure worked also
print(len(train_img))
print(len(train_targets))

train_x = np.array(train_img)
train_y = np.array(train_targets)

print(train_x.shape)
print(train_y.shape)

66
66


In [41]:
# Length of data
print(len(dataset))

# Split data!!!
size = len(dataset)
train_size = int(size * .80)
val_size = int(size * .20)


# split into train & validation
train_ds, val_ds = random_split(dataset, [train_size, val_size])


size == (train_size + val_size)

20580


True

In [23]:
train_x, val_x, train_y, val_y = train_test_split()

(tensor([[[0.0431, 0.0392, 0.0275,  ..., 0.6784, 0.6314, 0.5686],
          [0.0431, 0.0392, 0.0275,  ..., 0.6157, 0.5804, 0.5490],
          [0.0392, 0.0392, 0.0314,  ..., 0.4902, 0.4745, 0.4824],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 0.0235, 0.0235, 0.0235],
          [1.0000, 1.0000, 1.0000,  ..., 0.0235, 0.0235, 0.0235],
          [1.0000, 1.0000, 1.0000,  ..., 0.0275, 0.0275, 0.0275]],
 
         [[0.0353, 0.0314, 0.0235,  ..., 0.7765, 0.7294, 0.6471],
          [0.0353, 0.0314, 0.0235,  ..., 0.7137, 0.6706, 0.6314],
          [0.0314, 0.0314, 0.0275,  ..., 0.5922, 0.5686, 0.5686],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 0.0235, 0.0235, 0.0235],
          [1.0000, 1.0000, 1.0000,  ..., 0.0235, 0.0235, 0.0235],
          [1.0000, 1.0000, 1.0000,  ..., 0.0275, 0.0275, 0.0275]],
 
         [[0.0549, 0.0510, 0.0431,  ..., 0.6196, 0.5725, 0.5020],
          [0.0549, 0.0510, 0.0431,  ..., 0.5490, 0.5059, 0.4706],
          [0.0510, 0.0510, 0.0471,  ...,

In [7]:
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=2)

In [21]:
list(train_ds[0][0].size()) # need to resize these!!!!!!!

for x in train_loader:
    for y in x:
        y.reshape()

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\users\dsk02\desktop\python_projects\torch_dogs\env\lib\site-packages\torch\utils\data\_utils\worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\users\dsk02\desktop\python_projects\torch_dogs\env\lib\site-packages\torch\utils\data\_utils\fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "c:\users\dsk02\desktop\python_projects\torch_dogs\env\lib\site-packages\torch\utils\data\_utils\collate.py", line 83, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "c:\users\dsk02\desktop\python_projects\torch_dogs\env\lib\site-packages\torch\utils\data\_utils\collate.py", line 83, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "c:\users\dsk02\desktop\python_projects\torch_dogs\env\lib\site-packages\torch\utils\data\_utils\collate.py", line 55, in default_

# Initialize Model

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class StanfordDogs(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(), 
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10))
        
    def forward(self, xb):
        return self.network(xb)

In [10]:
model = StanfordDogs()

In [13]:
# Loss function & Optimizer

epochs = 8
max_lr = 0.01
# grad_clip = 0.1
# weight_decay = 1e-4

# cross entropy for classification
crit = nn.CrossEntropyLoss()

# george hotz says adam is the best
opt = optim.Adam(model.parameters(), lr=max_lr)

In [14]:
train_ds = train_ds.reshape(32,3,3)

AttributeError: 'Subset' object has no attribute 'reshape'

# Trainning Day

In [12]:
loss_arr = []

for i, data in enumerate(train_ds,0):
    # i in index, enumerate allows for tensor pass.. ignore and use
    
    # split data with inputs and labels
    inputs, labels = data
    
    # reshape?
    
    
    # zero the parameter gradients
    opt.zero_grad()
    
    #forward + backward + optimize 
    
    # get predictions
    outputs = model(inputs)
    
    # calculate how bad they are
    loss = crit(outputs, labels)
    
    # calculate the loss wrt to all the parameters in loss (dx)
    loss.backward()
    
    #update your parameters
    opt.step()
    
    # loss increase
    ls += loss.item()
    loss_arr.append(loss.item())
    

for i,loss in enumerate(loss_arr):
    print('Iteration {} & Loss {%d2}'.format(i,loss))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 3, 3, 3], but got 3-dimensional input of size [3, 375, 500] instead